# Notes
## Have commands as state machines
### use observer on them

each loop will go through the set of new/pending commands and then those commands can handle themselves


In [ ]:
class Command:
    
    def __init__(self):
        self.device = 'irig'
        self.state = 'new'
        self.target = 8
        self.action = 'watering'
        
    def set_state(self,new_state):
        self.state = new_state

In [124]:
import time
import random
import queue
from abc import ABC, abstractmethod

class PumpStateMachine:
    def __init__(self):
        print("Pump: Init")
        self.state = "Idle"
        self.is_running = False
        

    def set_state(self, new_state):
        print(f"Pump: from {self.state} to {new_state}")
        if new_state == "Running":
            self.is_running = True
        elif new_state == "Idle":
            self.is_running = False
        self.state = new_state

    def start_pump(self):
        print("Pump: Starting")
        self.set_state("Running")

    def stop_pump(self):
        self.set_state("Idle")
        print("Pump: Stopped")

    def tick(self):
        if self.state == "Running":
            print('Pump: Running')
            if not self.is_running:
                self.start_pump()

        elif self.state == "Idle":
            print('Pump: Idle')
            if self.is_running:
                self.stop_pump()

class ReservoirStateMachine:
    def __init__(self):
        print("Reservoir: Init")
        self.state = "Init"
        self.weight = self.read_weight()
        print(f'Reservoir: Starting with {self.weight} in res')
        
              
    def set_state(self, new_state):
        print(f"Reservoir: from {self.state} to {new_state}")
        if new_state == "Full":
            pass
        elif new_state == "Empty":
            pass
        self.state = new_state

    def read_weight(self):
        if self.state == "Init":
            fake_sense = random.randrange(50, 75)
            self.set_state('Full')
        else:
            fake_sense = self.weight#random.randrange(0, self.weight+1)
        return fake_sense

    def tick(self):
        reading = self.read_weight()
        self.weight = reading
        
        if self.state == 'Full':
            if self.weight <= 49:
                print('new weight under 10')
                self.set_state('Empty')
        elif self.state == "Empty":
            print("Irrigator: Im empty!!")

class Irrigator:
    
    def __init__(self,pump,res):
        self.pump = pump
        self.res = res
        self.target_weight = 0
        self.cumulative_weight_dispensed = 0
        self.state = "Idle"
        self.is_running = False
        
    def set_state(self, new_state):
        print(f"Irrigator: from {self.state} to {new_state}")
        if new_state == "Watering":
            self.is_running = True
        elif new_state == "Idle":
            self.is_running = False
        self.state = new_state

    def start_watering(self):
        print("Irrigator: Starting")
        if self.res.state != 'Empty':
            self.set_state("Watering")
            self.pump.start_pump()

    def stop_watering(self):
        self.set_state("Idle")
        print("Irrigator: Stopped")
    
    def set_target_weight(self, target_weight):
        if target_weight > 0:
            self.target_weight = self.res.weight - target_weight
            print(f"Irrigator: Target weight set to {self.target_weight} grams")
        else:
            print(f"Irrigator: can stop")
            self.target_weight = 0
    
    def __repr__(self):
        return f"Irrigator: Res Level:{self.res.weight}"
    
    def tick(self):

        
        if self.state == "Watering":
            print(f'Irrigator: Watering Plants to target of {self.target_weight}')
            
            if not self.is_running:
                self.is_running = True
                self.start_watering()

            if self.pump.state == 'Running': # simulate water coming out
                self.res.weight -= 1
             
            if self.res.state == 'Empty':
                print('Refill Reservoir')
                self.pump.stop_pump()
                self.stop_watering()
                self.set_target_weight(0)
                
            if self.target_weight >= self.res.weight:    
                print(f"Irrigator: Reached target Weight of {self.target_weight} with {self.res.weight}")
                self.pump.stop_pump()
                self.set_target_weight(0)
                self.stop_watering()
                
            self.res.tick()
            self.pump.tick()


        elif self.state == "Idle":
            print('Irrigator: Idle')
            if self.is_running:
                self.stop_watering()
        
# # Command interface
# class Command(ABC):
#     @abstractmethod
#     def execute(self):
#         pass
# 
# # Specific command for watering plants
# class WaterPlantsCommand(Command):
#     def __init__(self, irrigator, target):
#         self.irrigator = irrigator
#         self.target = target
# 
#     def execute(self):
#         self.irrigator.set_target_weight(self.target)
#         self.irrigator.start_watering()
# 
# # Specific command for turning on the fan
# class TurnOnFanCommand(Command):
#     def __init__(self, fan):
#         self.fan = fan
# 
#     def execute(self):
#         self.fan.turn_on()
#         
# class CommandHandler:
#     def __init__(self):
#         self.command_queue = queue.Queue()
# 
#     def add_command(self, command):
#         self.command_queue.put(command)
# 
#     def run(self):
#         while not self.command_queue.empty():
#             command = self.command_queue.get()
#             command.execute()
#             self.command_queue.task_done()

        
        
        
class IrigCommand:
    
    def __init__(self,irrigator):
        self.device = irrigator
        self.state = 'new'
        self.target = 8
        self.action = 'watering'
        
    def set_state(self,new_state):
        self.state = new_state        
        
    def __repr__(self):
        return f"Command: For {self.device}, target:{self.target}, action:{self.action}"
    
    def tick(self):
        self.device.tick()
        
irig = Irrigator(PumpStateMachine(),ReservoirStateMachine())
water = IrigCommand(irig)


while True:
    water.tick()
    time.sleep(1)
        
# command_handler = CommandHandler()
# # Populate the command handler with command objects
# command_handler.add_command(WaterPlantsCommand(irig, 8))
# command_handler.add_command(TurnOnFanCommand(fan))  # Assuming 'fan' is a control object for your fan
# 
# # Main control loop
# while True:
#     start_time = time.time()
#     
#     # Update the state machines
#     irig.tick()
# 
#     # Process commands
#     command_handler.run()
# 
#     time.sleep(1)
#     end_time = time.time()
#     print(f"total time taken this tick: {(end_time - start_time)*1000}ms")
#     print()


Pump: Init
Reservoir: Init
Reservoir: from Init to Full
Reservoir: Starting with 73 in res
Irrigator: Idle
Irrigator: Idle
Irrigator: Idle
Irrigator: Idle
Irrigator: Idle
Irrigator: Idle


KeyboardInterrupt: 

Sketch of control loop

check for commands

iterate through command queue
    add new commands
    parse existing commands
    clear completed commands
   
each device has its space for one active command at a time to protect from multiple commands being issues. Other commands can either be rejected or queued.

remember that the main loop operates in ticks. 

so lets say we are idling and r pi sends a watering command. We will need to:

1. Acknowledge command -> rpi
2. set a target for irrigator
3. tick
4. loop

So maybe the way this works is simply that that commands give simple instructions, and provide a means to communicate between r pi and pico. Basically the irigator as implemented takes care of itself.

so basically from a command stand point, we just need to issue the target to the irrigator. the irrigator needs to be blocked from receiving commands, and needs to let the "command" that it's done.

All devices tick on each loop -> is this efficient?

So the question now really is how do we link the irrigator class (and similar) to the command queue? Also how do i parse commands.



 


In [115]:


commands_list = [{
    'device':'irig',
    'command':'water_plant',
    'status':'New',
    'target':8
},
{
    'device':'fan',
    'command':'blow_air',
    'status':'New'
}]

completed_commands =[]

# Create a single instance of the StateMachine
irig = Irrigator(PumpStateMachine(),ReservoirStateMachine())
print('starting control loop\n')

while True:
    # Check the weight of the reservoir
    start_time = time.time()
    weight = irig.res.weight
    print(f"Current weight: {irig.res.weight} grams, target = {irig.target_weight}")
    
    
    for command in commands_list:

        if command['status'] == 'New':
            print('handle new command')
            if command['device'] == 'irig':
                command['status'] ='Pending'

        elif command['status'] == 'Pending':
            print(f'command {command} pending')
            command['status'] = 'Complete'
            # move command out of active list
            commands_list.remove(command)
            completed_commands.append(command)
        elif command['status'] == 'Failed':
            print('command failed')

        elif command['status'] == 'Complete':
            print('command complete')

        else:
            print('this command seems weird')

    # if weight > 50:
    #     if irig.state != 'Watering':
    #         irig.set_target_weight(8)
    #         irig.start_watering()
    # Update the state machine
    irig.tick()
    time.sleep(1)
    end_time = time.time()
    print(f"total time taken this tick: {(end_time - start_time)*1000}ms")
    print()

Pump: Init

Reservoir: Init
Reservoir: from Init to Full
Reservoir: Starting with 62 in res
starting control loop

Current weight: 62 grams, target = 0
handle new command
handle new command
Irrigator: Idle
total time taken this tick: 1003.594160079956ms

Current weight: 62 grams, target = 0
command {'device': 'irig', 'command': 'water_plant', 'status': 'Pending', 'target': 8} pending
Irrigator: Idle
total time taken this tick: 1004.8840045928955ms

Current weight: 62 grams, target = 0
handle new command
Irrigator: Idle
total time taken this tick: 1000.4048347473145ms

Current weight: 62 grams, target = 0
handle new command
Irrigator: Idle
total time taken this tick: 1000.1552104949951ms

Current weight: 62 grams, target = 0
handle new command
Irrigator: Idle


KeyboardInterrupt: 

In [116]:
completed_commands

[{'device': 'irig',
  'command': 'water_plant',
  'status': 'Complete',
  'target': 8}]

In [117]:
commands_list

[{'device': 'fan', 'command': 'blow_air', 'status': 'New'}]

I wonder if these state machines needs to take commands
how should a pump be told how long to run for?

I guess for the pump i want to set a number of grams (or proxy of grams) to dispense, and on every tick check if that cumulative weight is reached.
    Not sure i like the idea of it being coupled to soil moisture sensor.
    I'll need the reservoir starting weight and then store it (or the target weight) and then check to see when it crosses that threshold
    This will instruct the pump to stop dispensing water.

For the fan i probably want it to be timed



In [28]:
class StateMachine:
    def __init__(self):
        print("Pump: Init\n")
        self.state = "Idle"
        self.is_running = False

    def set_state(self, new_state):
        print(f"Pump: from {self.state} to {new_state}")
        if new_state == "Running":
            self.is_running = True
        elif new_state == 'Idle':
            self.is_running = False
        self.state = new_state
        
    def start_pump(self):
        self.set_state("Running")
        print(f"Pump: Started")

    def stop_pump(self):
        self.set_state("Idle")
        print(f"Pump: Stopped")

    def take_sensor_reading(self):
        fake_sense = random.randrange(0,100)
        return fake_sense

    def tick(self):
        if self.state == "Running":
            if not self.is_running:
                self.start_pump()

            reading = self.take_sensor_reading()
            if reading >= 75:
                print(f"Pump: Enough Water: {reading}")
                self.stop_pump()

        elif self.state == "Idle":
            pass
            # reading = self.take_sensor_reading()
            # 
            # if reading <= 10:
            #     print(f"Pump: Not Enough Water: {reading}")
            #     self.start_pump()

In [34]:
def get_weight_reading():
    fake_sense = random.randrange(0,100)
    return fake_sense

In [ ]:

# Create a single instance of the StateMachine
# pump_sm = PumpStateMachine()
# reservoir_sm = ReservoirStateMachine()
# print('starting control loop\n')
# while True:
#     # Check the weight of the reservoir
#     weight = reservoir_sm.weight
#     print(f"Current weight: {weight} grams")
#     
#     if weight > 50:
#         if pump_sm.state != 'Running':
#             pump_sm.set_target_weight(8)
#             pump_sm.start_pump()
#     # Update the state machine
#     pump_sm.tick()
#     reservoir_sm.tick()
#     time.sleep(1)  # Adjust the loop interval as needed
#     print()

In [ ]:
class Command:
    def __init__(self,target):
        self.target = target

In [32]:

# sample main loop of overall system

moisture = 'dry'
water_level = 'full'


new_commands = [{'pump':'on','ack':None},{'fan':'on','ack':None}]
pending_commands = []
completed_commands = []

commands_v2 = [{
    'device':'pump',
    'command':'water_out',
    'status':'New',
    'target':50
},
{
    'device':'fan',
    'command':'blow_air',
    'status':'New'
}]
#[{'pump':'on','status':'New'},{'fan':'on','status':'New'}]

pump = StateMachine()
i=0
while i < 10:
    start_time = time.time()
    pump.tick()
    print('tick start')
    
    # print('check sensors')
    # print('\tcheck moisture status -> dry or wet', moisture)
    # print('\tcheck water level -> full, low, empty', water_level)
    
    # instead of having 3 loops, maybe the command should also have status
    # command statuses = ['New','Pending','Failed','Completed']
    # for command in new_commands:
    #     print('figuring out new commands, move to pending')
    #     
    # for command in pending_commands:
    #     print('checking up on commands being serviced, move to complete when done')
    #     
    # for command in completed_commands:
    #     print('report back on completed commands, then clear them out')
    #     
    
    for command in commands_v2:
        
        if command['status'] == 'New':
            print('handle new command')
            if command['device'] == 'pump':
                pump.set_state('Running')
                command['status'] ='Pending'
        elif command['status'] == 'Pending':
            print(f'command {command} pending')
        elif command['status'] == 'Failed':
            print('command failed')
        elif command['status'] == 'Complete':
            print('command failed')
        else:
            print('this command seems weird')
    
    
    
    
    #pump = input('should we pump')

    if pump == 'y':
        print('we should start pumping, change the status to pumping')
        print('put command in pending status')
        
        
    
        
    print('check pump status')
    
    print('report activity to server')
    i += 1
    time.sleep(0.2)
    end_time = time.time()
    print("total time taken this tick: ", (end_time - start_time)*1000)
    print()


Pump: Init

tick start
handle new command
Pump: from Idle to Running
handle new command
check pump status
report activity to server
total time taken this tick:  201.46417617797852

tick start
command {'device': 'pump', 'command': 'water_out', 'status': 'Pending'} pending
handle new command
check pump status
report activity to server
total time taken this tick:  201.9200325012207

tick start
command {'device': 'pump', 'command': 'water_out', 'status': 'Pending'} pending
handle new command
check pump status
report activity to server
total time taken this tick:  202.3789882659912

tick start
command {'device': 'pump', 'command': 'water_out', 'status': 'Pending'} pending
handle new command
check pump status
report activity to server
total time taken this tick:  204.26297187805176

tick start
command {'device': 'pump', 'command': 'water_out', 'status': 'Pending'} pending
handle new command
check pump status
report activity to server
total time taken this tick:  204.57196235656738

tick star